In [1]:
from IPython import get_ipython

%pip install hvplot
%pip install panel
%pip install seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [3]:
df = pd.read_csv("FloodPrediction.csv")

FileNotFoundError: ignored

In [ ]:
df

## (0) Some minor data preprocessing

In [ ]:
# Fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
#df['gdp_per_capita'] = np.where(df['population']!= 0, df['gdp']/ df['population'], 0)

In [ ]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

## (1) CO2 emission over time by continent

In [ ]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=1948, end=2014, step=5, value=1948)
year_slider

In [ ]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['Rainfall', 'Cloud_Coverage','Wind_Speed','Relative_Humidity', ],
    button_type='success'
)

In [ ]:
continents = ['Barisal', 'Bogra', 'Chandpur', 'Chittagong (IAP-Patenga)', 'Comilla',
       "Cox's Bazar", 'Dhaka', 'Faridpur', 'Feni',  'Jessore', 'Khulna',
       'Madaripur', 'Mongla', 'Mymensingh', 'Patuakhali',
       'Rajshahi', 'Rangamati', 'Satkhira',
       'Sitakunda', 'Srimangal', 'Sylhet', 'Teknaf']

co2_pipeline = (
    idf[
        (idf.Year <= year_slider) &
        (idf.Station_Names.isin(continents))
    ]
    .groupby(['Station_Names', 'Year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [ ]:
co2_pipeline

In [ ]:
co2_plot = co2_pipeline.hvplot(x = 'Year', by='Station_Names', y=yaxis_co2,line_width=2, title="")
co2_plot

## (2) Table - CO2 emission over time by continent

In [ ]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
co2_table

## (3) CO2 vs GDP scatterplot

## (4) Bar chart with CO2 sources by continent

In [ ]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['Rainfall', 'Cloud_Coverage','Wind_Speed','Relative_Humidity', ],
    button_type='success'
)

continents_excl_world = ['Barisal', 'Bogra', 'Chandpur', 'Chittagong (IAP-Patenga)', 'Comilla',
       "Cox's Bazar", 'Dhaka', 'Faridpur', 'Feni',  'Jessore', 'Khulna',
       'Madaripur', 'Mongla', 'Mymensingh', 'Patuakhali',
       'Rajshahi', 'Rangamati', 'Satkhira',
       'Sitakunda', 'Srimangal', 'Sylhet', 'Teknaf']

co2_source_bar_pipeline = (
    idf[
        (idf.Year == year_slider) &
        (idf.Station_Names.isin(continents_excl_world))
    ]
    .groupby(['Year', 'Station_Names'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [ ]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar',
                                                     x='Station_Names',
                                                     y=yaxis_co2_source,
                                                     title='')
co2_source_bar_plot

## Creating Dashboard

In [ ]:
import panel as pn
import numpy as np
import holoviews as hv

pn.extension(sizing_mode = 'stretch_width')

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(

    title='Correlation of flood and flood-based parameters',
    sidebar=[pn.pane.Markdown("# AN INTERACTIVE FLOOD FORECASTING TOOL WITH ENSEMBLE-BASED MACHINE LEARNING MODEL: A BANGLADESH PERSPECTIVE"),
             pn.pane.Markdown("#### Bangladesh is prone to frequent floods due to its low-lying location and vulnerability to monsoon rains, melting snow from the Himalayas, deforestation, and local rainfall. An ensemble-based machine learning approach is proposed for flood forecasting using meteorological and remote sensing data."),

             pn.pane.Markdown("## Settings"),
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, co2_plot.panel(width=800), margin=(0,30)),co2_table.panel(width=600)),

                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=1400))],
    accent_base_color="#6ED6F9",
    header_background="#6ED6F9",
)
# template.show()
template.servable();

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
